<a href="https://colab.research.google.com/github/DarkLord-13/Machine-Learning-01/blob/main/Fine_Tuning_LLama_3_8B_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Improvements:
1. prompt-engineering

First we check the GPU version available in the environment and install specific dependencies that are compatible with the detected GPU to prevent version conflicts.

In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" # installs the unsloth library directly from github repo, particularly for google colab environment
if major_version >= 8: # if the major version of cuda is less than 8
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else: # --no-deps tells pip to install only the particular package and not any dependencies
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

Next we need to prepare to load a range of quantized language models, including a new 15 trillion token LLama-3 model, optimized for memory efficiency with 4-bit quantization.


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Llama 3 is up to 8k
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

fourbit_models = [ # list of all the 4-bit models
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained( # loads the model and tokenizer of the particular model we choose
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Llama-3 70b also works (just change the model name)
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.




---



Next, we integrate LoRA adapters into our model, which allows us to efficiently update just a fraction of the model's parameters, enhancing training speed and reducing computational load.

In [3]:
model = FastLanguageModel.get_peft_model(
    model, # base model that we are fine-tuning, here-> llama-3
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128 (rank of the low-rank adaptation matrices, higher values means better adaption but more cost as well)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # increased values means more influence of the low-rank matrices on the overall model
    lora_dropout = 0, # no regularization on the adaptation matrices
    bias = "none", # specifies how the biases are used in the model, here 'none'
    use_gradient_checkpointing = "unsloth", # instead of storing all the weights in RAM during the whole forward and backward pass, here the weights are saved and the intermediate weights are recomputed saving memory usage
    random_state = 3407, # for replicating the same fine-tuning process
    use_rslora = False, #
    loftq_config = None,
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import json
from datasets import Dataset

# Create the dataset
qa_dataset = [
    {
        "instruction": "What are the working hours?",
        "input": "",
        "output": "The standard working hours are from 9 AM to 6 PM, Monday to Friday."
    },
    {
        "instruction": "What is the dress code?",
        "input": "",
        "output": "The dress code is business casual from Monday to Thursday, and casual on Fridays."
    },
    {
        "instruction": "How do I apply for leave?",
        "input": "",
        "output": "You can apply for leave through the company's HR portal. Ensure to get it approved by your manager."
    },
    {
        "instruction": "What benefits are provided?",
        "input": "",
        "output": "The company provides health insurance, retirement benefits, and performance bonuses."
    },
    {
        "instruction": "How can I get my employee ID?",
        "input": "",
        "output": "You will receive your employee ID on your first day from the HR department."
    },
    {
        "instruction": "What is the company's policy on remote work?",
        "input": "",
        "output": "The company offers flexible remote work options, but it must be approved by your manager."
    },
    {
        "instruction": "How do I access the company's intranet?",
        "input": "",
        "output": "You can access the company's intranet using your employee credentials through the internal network."
    },
    {
        "instruction": "What should I do if I have a problem with my payroll?",
        "input": "",
        "output": "If you have a payroll issue, contact the payroll department via the HR portal or email payroll@company.com."
    },
    {
        "instruction": "Are there any employee training programs?",
        "input": "",
        "output": "Yes, the company offers various training programs for skill development and career advancement. Check the HR portal for more details."
    },
    {
        "instruction": "How can I update my personal information?",
        "input": "",
        "output": "You can update your personal information through the HR portal under the 'Profile' section."
    },
    {
        "instruction": "What is the procedure for reporting harassment?",
        "input": "",
        "output": "To report harassment, contact the HR department directly or use the anonymous reporting feature on the HR portal."
    },
    {
        "instruction": "How do I find out about job openings within the company?",
        "input": "",
        "output": "Internal job openings are posted on the company's intranet under the 'Careers' section."
    },
    {
        "instruction": "What is the company's travel policy?",
        "input": "",
        "output": "The travel policy includes guidelines for booking travel, expense reimbursement, and travel safety. Details can be found on the HR portal."
    },
    {
        "instruction": "How can I contact the IT support team?",
        "input": "",
        "output": "You can contact IT support via the helpdesk portal or by calling the IT support hotline at 123-456-7890."
    },
    {
        "instruction": "What is the company's policy on overtime?",
        "input": "",
        "output": "Overtime must be pre-approved by your manager. Overtime pay is calculated according to state and federal laws."
    },
    {
        "instruction": "How can I participate in the company's wellness programs?",
        "input": "",
        "output": "Information on wellness programs, including how to sign up, is available on the HR portal under 'Wellness Programs'."
    },
    {
        "instruction": "What resources are available for mental health support?",
        "input": "",
        "output": "The company offers mental health support through Employee Assistance Programs (EAP). Contact HR for more information."
    },
    {
        "instruction": "What is the process for getting reimbursed for work-related expenses?",
        "input": "",
        "output": "Submit your receipts and expense report through the expense management system on the HR portal."
    },
    {
        "instruction": "How can I access my pay stubs?",
        "input": "",
        "output": "Pay stubs are available online through the payroll section of the HR portal."
    },
    {
        "instruction": "What are the company's holiday policies?",
        "input": "",
        "output": "The company observes all major national holidays. A detailed list of holidays is available on the HR portal."
    }
]


# Convert the list to a Dataset
dataset = Dataset.from_list(qa_dataset)

EOS_TOKEN = tokenizer.eos_token  # Replace with the actual EOS token from the tokenizer

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# gets a dataset called examples with 3 keys and their corresponsing values as a list
def formatting_prompts_func(examples):
    instructions = examples["instruction"] # list of all the instructions
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN  # Add EOS token to end of the text
        texts.append(text)
    return {"text": texts}

# Apply the formatting function to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

# Print the formatted dataset to verify
print(dataset)


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 20
})


##Example walkthrough:

Let's assume examples contains the following data:

```
examples = {
    "instruction": ["Translate to French.", "Summarize the text."],
    "input": ["Hello, how are you?", "This is a story about a fox."],
    "output": ["Bonjour, comment ça va?", "A story about a fox."]
}

```

1. Extracted Lists:


*   Instructions: ["Translate to French.", "Summarize the text."]
*   Inputs: ["Hello, how are you?", "This is a story about a fox."]
*   Outputs: ["Bonjour, comment ça va?", "A story about a fox."]

2. Formatted Texts:

*   For the first example:

```
"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Translate to French.

### Input:
Hello, how are you?

### Response:
Bonjour, comment ça va?" + EOS_TOKEN
```


*   For the second example:

```
"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the text.

### Input:
This is a story about a fox.

### Response:
A story about a fox." + EOS_TOKEN
```

3. Resulting List:

```
[
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Translate to French.

    ### Input:
    Hello, how are you?

    ### Response:
    Bonjour, comment ça va?" + EOS_TOKEN,
    
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Summarize the text.

    ### Input:
    This is a story about a fox.

    ### Response:
    A story about a fox." + EOS_TOKEN
]
```

This transformation is applied to each batch in the dataset, ensuring that all examples are consistently formatted according to the template.




<a name="Train"></a>
### Train the model
- We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.
- At this stage, we're configuring our model's training setup, where we define things like batch size and learning rate, to teach our model effectively with the data we have prepared.

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # processors used for data processing
    packing = False, # Can make training 5x faster for short sequences. We will combine multiple examples to fill the model's memory and make training more efficient instead of feeding examples individually.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # increase this to make the model learn better
        #num_train_epochs=4,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01, # penalizes the model if weight update is too large, prevents overfitting
        lr_scheduler_type = "linear", # decreases the lr linearly
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/20 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [6]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.605 GB of memory reserved.


In [7]:
# We're now starting the actual training of our model, which will print out some statistics showing us how well it learns
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20 | Num Epochs = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.885900
2,2.926500
3,2.843300
4,2.557100
5,2.197800
6,1.735300
7,1.305700
8,0.987600
9,0.676800
10,0.648100


In [8]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

261.8114 seconds used for training.
4.36 minutes used for training.
Peak reserved memory = 6.1 GB.
Peak reserved memory for training = 0.495 GB.
Peak reserved memory % of max memory = 41.362 %.
Peak reserved memory for training % of max memory = 3.356 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [9]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "How can I get my employee ID?", # instruction
        "", # input
        "", # output - leave this blank for generation
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nHow can I get my employee ID?\n\n### Input:\n\n\n### Response:\nYou will receive your employee ID on your first day from the HR department.<|end_of_text|>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [18]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "How diverse is the culture of this company?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
How diverse is the culture of this company?

### Input:


### Response:
The company values diversity and has various programs to promote inclusion.<|end_of_text|>


In [19]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "is plagiarism allowed in the company?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
is plagiarism allowed in the company?

### Input:


### Response:
Plagiarism is not allowed in the company. It is considered a serious offense and may result in disciplinary action.<|end_of_text|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [11]:
model.save_pretrained("lora_model") # Local saving

In [14]:
model.push_to_hub("nishantkumar1/lora_model_POC_1", token = "") # Online saving

README.md:   0%|          | 0.00/580 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/nishantkumar1/lora_model_POC_1


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [15]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

# alpaca_prompt = You MUST run cells from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "can we bring kids to our offices and leave them unsupervised", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\ncan we bring kids to our offices and leave them unsupervised\n\n### Input:\n\n\n### Response:\nYes, but it must be pre-approved by your manager.<|end_of_text|>']

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
'''if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")'''

We're preparing to save our trained model in a more compact format and then upload it to a cloud platform, which allows us to use less storage and computational power.

In [ ]:
'''# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")'''

We're ready to compress our model using various quantization methods to make it leaner and then upload it to the cloud for easy sharing and access.

In [ ]:
'''# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")'''

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, join their [Discord](https://discord.gg/u54VK8m8tk) channel!